In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['FRAP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [13]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [14]:
list(zip(longforms, counts))

[('fluorescence recovery after photobleaching', 170),
 ('ferric reducing antioxidant power', 114),
 ('ferric reducing ability of plasma', 48),
 ('fluorescence recovery after photo bleaching', 13),
 ('ferric reducing antioxidant potential', 10),
 ('fkbp12 rapamycin associated protein', 10),
 ('ferric ion reducing antioxidant power', 9),
 ('ferric reducing antioxidant power assay', 6),
 ('ferric reducing activity in plasma', 5),
 ('fkbp rapamycin associated protein', 4),
 ('ferric reduction antioxidant power', 3),
 ('fkbp 12 rapamycin associated protein', 2),
 ('antioxidant food', 2)]

In [16]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-30 05:19:01] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [17]:
result = [grounding_map, names, pos_labels]

In [18]:
result

[{'antioxidant food': 'ungrounded',
  'ferric ion reducing antioxidant power': 'ungrounded',
  'ferric reducing ability of plasma': 'ungrounded',
  'ferric reducing activity in plasma': 'ungrounded',
  'ferric reducing antioxidant potential': 'ungrounded',
  'ferric reducing antioxidant power': 'ungrounded',
  'ferric reducing antioxidant power assay': 'ungrounded',
  'ferric reduction antioxidant power': 'ungrounded',
  'fkbp 12 rapamycin associated protein': 'ungrounded',
  'fkbp rapamycin associated protein': 'HGNC:3942',
  'fkbp12 rapamycin associated protein': 'HGNC:3942',
  'fluorescence recovery after photo bleaching': 'MESH:D036681',
  'fluorescence recovery after photobleaching': 'MESH:D036681'},
 {'HGNC:3942': 'MTOR',
  'MESH:D036681': 'Fluorescence Recovery After Photobleaching'},
 ['HGNC:3942', 'MESH:D036681']]

In [19]:
grounding_map, names, pos_labels = [{'antioxidant food': 'ungrounded',
  'ferric ion reducing antioxidant power': 'ungrounded',
  'ferric reducing ability of plasma': 'ungrounded',
  'ferric reducing activity in plasma': 'ungrounded',
  'ferric reducing antioxidant potential': 'ungrounded',
  'ferric reducing antioxidant power': 'ungrounded',
  'ferric reducing antioxidant power assay': 'ungrounded',
  'ferric reduction antioxidant power': 'ungrounded',
  'fkbp 12 rapamycin associated protein': 'ungrounded',
  'fkbp rapamycin associated protein': 'HGNC:3942',
  'fkbp12 rapamycin associated protein': 'HGNC:3942',
  'fluorescence recovery after photo bleaching': 'MESH:D036681',
  'fluorescence recovery after photobleaching': 'MESH:D036681'},
 {'HGNC:3942': 'MTOR',
  'MESH:D036681': 'Fluorescence Recovery After Photobleaching'},
 ['HGNC:3942', 'MESH:D036681']]

In [20]:
excluded_longforms = []

In [21]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [22]:
additional_entities = {}

In [23]:
unambiguous_agent_texts = {}

In [24]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [22]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [23]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [24]:
intersection1

[]

In [25]:
intersection2

[]

In [26]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

KeyboardInterrupt: 

In [ ]:
names.update(additional_entitie)

In [25]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 05:21:55] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 05:21:59] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9648223288588029 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [26]:
classifier.stats

{'label_distribution': {'MESH:D036681': 121,
  'HGNC:3942': 11,
  'ungrounded': 137},
 'f1': {'mean': 0.964822, 'std': 0.008586},
 'precision': {'mean': 0.973476, 'std': 0.026447},
 'recall': {'mean': 0.962393, 'std': 0.023435},
 'ungrounded': {'f1': {'mean': 0.981813, 'std': 0.016273},
  'pr': {'mean': 0.985714, 'std': 0.028571},
  'rc': {'mean': 0.979064, 'std': 0.027713}},
 'HGNC:3942': {'f1': {'mean': 0.853333, 'std': 0.129271},
  'pr': {'mean': 0.833333, 'std': 0.210819},
  'rc': {'mean': 0.933333, 'std': 0.133333}},
 'MESH:D036681': {'f1': {'mean': 0.975326, 'std': 0.007673},
  'pr': {'mean': 0.975333, 'std': 0.020149},
  'rc': {'mean': 0.976615, 'std': 0.030929}}}

In [27]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [28]:
disamb.dump(model_name, results_path)

In [29]:
print(disamb.info())

Disambiguation model for FRAP

Produces the disambiguations:
	Fluorescence Recovery After Photobleaching*	MESH:D036681
	MTOR*	HGNC:3942

Class level metrics:
--------------------
Grounding                                 	Count	F1     
                                Ungrounded	137	0.98181
Fluorescence Recovery After Photobleaching*	121	0.97533
                                      MTOR*	 11	0.85333

Weighted Metrics:
-----------------
	F1 score:	0.96482
	Precision:	0.97348
	Recall:		0.96239

* Positive labels
See Docstring for explanation



In [30]:
model_to_s3(disamb)